
# Clase 6 - Fundamentos de IA (IBM)
## Tema: Limpieza y Transformación de Datos con Python (pandas)

En esta clase aprenderemos a preparar datos antes de analizarlos.  
Este proceso es fundamental porque **los datos sin limpiar pueden llevar a conclusiones erróneas**.

---

### Objetivos de Aprendizaje
1. Comprender la importancia de la limpieza de datos en el ciclo de vida de la analítica.
2. Identificar problemas comunes en datasets reales.
3. Aplicar técnicas de limpieza con `pandas`.
4. Realizar transformaciones básicas para mejorar la calidad de los datos.
5. Resolver un caso práctico de negocio: **Café de Barrio**.

---

### Preguntas iniciales para reflexionar
- ¿Qué recuerdas de la clase anterior?
- ¿Qué esperas aprender hoy?
- ¿Qué dudas tienes antes de comenzar?


## 1. Importación de Librerías y Configuración Inicial

In [ ]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random

# Configuración para mostrar más información en las tablas
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 50)

print("Librerías cargadas correctamente.")



## 2. Generación de un Dataset de Ejemplo (Simulado)

Para practicar, vamos a **generar un dataset grande** (300 registros) con diferentes problemas:
- Valores faltantes.
- Duplicados.
- Fechas en distintos formatos.
- Emails repetidos.
- Precios negativos o extremadamente altos.
- Texto en diferentes capitalizaciones.

Así podremos aplicar todas las técnicas de limpieza en un solo caso.


In [ ]:

# Función para generar datos de ejemplo
np.random.seed(42)
random.seed(42)

clientes = ["Ana", "Luis", "Carla", "Pedro", "Maria", "Sofia", "Carlos", "Laura", "Diego", "Valentina"]
emails = [c.lower() + "@mail.com" for c in clientes]

def generar_fecha():
    formatos = ["%Y-%m-%d", "%d/%m/%Y", "%b %d, %Y"]
    fecha = pd.to_datetime("2024-01-01") + pd.to_timedelta(np.random.randint(0, 365), unit="D")
    return fecha.strftime(random.choice(formatos))

data = {
    "cliente": [random.choice(clientes + [None]) for _ in range(300)],
    "fecha": [generar_fecha() if random.random() > 0.05 else None for _ in range(300)],
    "email": [random.choice(emails) for _ in range(300)],
    "precio": [random.choice([random.randint(-100, 2000), None, 999999]) for _ in range(300)],
    "satisfaccion": [random.choice([1,2,3,4,5,None]) for _ in range(300)]
}

df = pd.DataFrame(data)
df.head(10)


## 3. Inspección Inicial del Dataset

In [ ]:
df.info()

In [ ]:
df.describe(include='all')

In [ ]:
df.isnull().sum()


## 4. Tratamiento de Valores Faltantes

Estrategias posibles:
1. Eliminar filas con valores nulos.
2. Rellenar con un valor estadístico (media, mediana, moda).
3. Rellenar con un valor fijo.
4. Usar métodos de propagación (`ffill` o `bfill`).


In [ ]:

# Eliminar filas con valores faltantes en 'cliente'
df_dropna = df.dropna(subset=['cliente'])
print("Tamaño original:", df.shape)
print("Después de eliminar:", df_dropna.shape)

# Rellenar NaN en precio con la mediana
df['precio'] = df['precio'].fillna(df['precio'].median())

# Rellenar NaN en satisfacción con 3 (neutral)
df['satisfaccion'] = df['satisfaccion'].fillna(3)

df.head(10)


## 5. Eliminación de Duplicados

In [ ]:

# Buscar duplicados en email
duplicados = df[df.duplicated(subset=['email'], keep=False)]
print("Duplicados detectados:", len(duplicados))

# Eliminar duplicados
df = df.drop_duplicates(subset=['email'])
df.head()


## 6. Manejo de Inconsistencias de Formato en Fechas

In [ ]:

# Convertir columna fecha a datetime
df['fecha'] = pd.to_datetime(df['fecha'], errors='coerce')
df[['cliente', 'fecha']].head(10)


## 7. Manejo de Valores Atípicos (Outliers)

In [ ]:

# Detección con boxplot
plt.boxplot(df['precio'])
plt.title("Boxplot de precios")
plt.show()

# Filtrar precios negativos o extremadamente altos
df = df[(df['precio'] > 0) & (df['precio'] < 10000)]
df.describe()


## 8. Transformaciones Básicas

In [ ]:

# Normalizar nombres de clientes a mayúsculas
df['cliente'] = df['cliente'].str.upper()

# Crear nueva columna de rango de satisfacción
df['satisfaccion_categoria'] = df['satisfaccion'].replace({1:"Muy Baja",2:"Baja",3:"Media",4:"Alta",5:"Muy Alta"})

# Crear nueva columna: logaritmo del precio
df['precio_log'] = np.log(df['precio'] + 1)

df.head(10)


## 9. Ejemplos Adicionales de Transformaciones

In [ ]:

# Extraer año y mes de la fecha
df['anio'] = df['fecha'].dt.year
df['mes'] = df['fecha'].dt.month

# Contar cuántos clientes por mes
clientes_mes = df.groupby('mes')['cliente'].count()
print(clientes_mes)

# Agrupar satisfacción promedio por año
satisfaccion_anual = df.groupby('anio')['satisfaccion'].mean()
print(satisfaccion_anual)


## 10. Caso Práctico: Café de Barrio

In [ ]:

# Simulación de datos de Café de Barrio
np.random.seed(123)
cafe = pd.DataFrame({
    "mes": pd.date_range("2024-01-01", periods=12, freq="M"),
    "temperatura": np.random.randint(18, 35, 12),
    "ventas": np.random.randint(200, 1000, 12),
    "publicidad": np.random.randint(100, 500, 12),
    "empleados": np.random.randint(3, 10, 12),
    "satisfaccion": np.random.randint(1, 6, 12)
})
cafe.head()


In [ ]:

# Correlación temperatura vs ventas
print("Correlación:", cafe['temperatura'].corr(cafe['ventas']))
plt.scatter(cafe['temperatura'], cafe['ventas'])
plt.xlabel("Temperatura (°C)")
plt.ylabel("Ventas")
plt.title("Relación temperatura - ventas")
plt.show()



## 11. Cierre y Reflexión

- ¿Qué fue lo más útil de la clase?  
- ¿Qué parte te costó más?  
- ¿Qué te gustaría repasar o reforzar?  

👉 Con esta práctica, entendimos cómo los **datos crudos** pueden transformarse en **datos útiles para la toma de decisiones**.
